In [1]:
import pandas as pd
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets
from multiprocessing import cpu_count
import pymc as pm
import numpy as np
from estival.wrappers import pymc as epm
from tbdynamics.utils import round_sigfig

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"

In [3]:
params = {
    # "contact_rate": 0.007255277552812773,
    # "start_population_size": 2200000,
    # "rr_infection_latent": 0.4435087078206438,
    # "rr_infection_recovered": 0.3726339126072788,
    # "progression_multiplier": 1.71734470040371,
    # "seed_time": 1810.0,
    # "seed_num": 70.0,
    # "seed_duration": 10.0,
    # "smear_positive_death_rate": 0.3789083132417181,
    # "smear_negative_death_rate": 0.023640460692223537,
    # "smear_positive_self_recovery": 0.2064856493052769,
    # "smear_negative_self_recovery": 0.11108908197816095,
    # "screening_scaleup_shape": 0.07019338853789103,
    # "screening_inflection_time": 1994.2958903597957,
    # "screening_end_asymp": 0.5881461121229823,
    # "detection_reduction": 0.7542772690107872,
}


bcm = get_bcm(params)

In [4]:
num_cores = cpu_count()
# Set the number of workers for parallel optimization
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class, num_workers=num_cores, ci=0.67)
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params

{'contact_rate': 0.012779205746969804,
 'start_population_size': 2345066.9832788385,
 'rr_infection_latent': 0.25316690520109597,
 'rr_infection_recovered': 0.4451005875083772,
 'progression_multiplier': 1.0708513643694917,
 'seed_time': 1824.295324327715,
 'seed_num': 74.5365151387086,
 'seed_duration': 11.562554112914142,
 'smear_positive_death_rate': 0.3656564026174954,
 'smear_negative_death_rate': 0.021809618134281075,
 'smear_positive_self_recovery': 0.21686817836177807,
 'smear_negative_self_recovery': 0.1582026232555177,
 'screening_scaleup_shape': 0.07075879817692829,
 'screening_inflection_time': 2005.0852566582118,
 'screening_end_asymp': 0.6530801407246749,
 'detection_reduction': 0.8302168594940716}

In [5]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

In [6]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [7]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [8]:
plot_model_vs_actual(derived_df_0, targets['prevalence_pulmonary'], 'prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')

In [9]:
plot_model_vs_actual(
    derived_df_0, targets['incidence'], "incidence", "Incidence", "Modelled vs Data"
)

In [10]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [11]:
# from autumn.infrastructure.remote import springboard
# rts = springboard.task.RemoteTaskStore()
# rts.cd('projects/tbdynamics/s10041_calibration')
# rts.ls()
# mt = rts.get_managed_task('2024-04-10T1335-d50k-t20k-b20k')
# mt.download_all()

In [12]:
derived_df_0['mortality'].plot()

In [13]:
derived_df_0['mortality_raw'].plot()